## <center style = "font-family:Arial">Amazon Book Data Analysis</center>

# <center>Import Libraries</center>



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings('ignore')
print('Setup Complete!')

Setup Complete!


**Note that some author names are not registered with Amazon, so you need to apply extra find for those authors. In the below cell code, you would find nested if-else conditions for author names, which are to extract the author/publication names.**

# <center>Web Scraping</center>

In [2]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.com/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

**The below code cell will perform the following functions:**

* Call the get_data function inside a for loop.
* The for loop will iterate over this function starting from 1 till the number of pages+1.
* Since the output will be a nested list, you would first flatten the list and then pass it to the DataFrame.
* Finally, save the dataframe as a CSV file.

In [3]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

# <center> Reading CSV File</center>

In [4]:
df = pd.read_csv("amazon_products.csv")

In [5]:
df.shape

(100, 5)

**The shape of the dataframe reveals that there are 100 rows and 5 columns in your CSV file.**


In [6]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,All American Christmas,Rachel Campos-Duffy,4.5 out of 5 stars,42,$18.77
1,"The Real Anthony Fauci: Bill Gates, Big Pharma...",Robert F. Kennedy Jr.,4.9 out of 5 stars,173,$17.59
2,The 1619 Project: A New Origin Story,Nikole Hannah-Jones,4.4 out of 5 stars,63,$24.99
3,Big Shot Diary of a Wimpy Kid Book 16,Jeff Kinney,4.8 out of 5 stars,"1,695",$9.00
4,Welcome to Dunder Mifflin: The Ultimate Oral H...,Brian Baumgartner,5.0 out of 5 stars,6,$20.45


In [7]:
df.tail()

,Book Name,Author,Rating,Customers_Rated,Price
95,The Mountain Is You: Transforming Self-Sabotag...,Brianna Wiest,4.8 out of 5 stars,"1,700",$16.69
96,Interesting Stories For Curious People: A Coll...,Bill O'Neill,4.5 out of 5 stars,"10,757",$8.19
97,The Unofficial Harry Potter Cookbook: From Cau...,Dinah Bucholz,4.7 out of 5 stars,"24,751",$9.99
98,Woke Up This Morning: The Definitive Oral Hist...,Michael Imperioli,4.4 out of 5 stars,161,$21.99
99,The Far Side® 2022 Off-The-Wall Calendar,Gary Larson,4.7 out of 5 stars,92,$14.39


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Book Name        100 non-null    object
 1   Author           100 non-null    object
 2   Rating           100 non-null    object
 3   Customers_Rated  100 non-null    object
 4   Price            100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


**preprocessing on the ratings, customers_rated, and price column.**
* Since you know the ratings are out of 5, you can keep only the rating and remove the extra part from it.
* From the customers_rated column, remove the comma.
* From the price column, remove the rupees symbol, comma, and split it by dot.
* Finally, convert all the three columns into integer or float.

# <center>Data Cleaning</center>

In [9]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
df['Rating'] = pd.to_numeric(df['Rating'])
df["Price"] = df["Price"].str.replace('$', '')
df["Price"] = df["Price"].str.replace(',', '')
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])
df['Price'] = df['Price'].astype(int)
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,All American Christmas,Rachel Campos-Duffy,4.5,42,18
1,"The Real Anthony Fauci: Bill Gates, Big Pharma...",Robert F. Kennedy Jr.,4.9,173,17
2,The 1619 Project: A New Origin Story,Nikole Hannah-Jones,4.4,63,24
3,Big Shot Diary of a Wimpy Kid Book 16,Jeff Kinney,4.8,1695,9
4,Welcome to Dunder Mifflin: The Ultimate Oral H...,Brian Baumgartner,5.0,6,20


### Let's verify the data types of the DataFrame.

In [10]:
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price                int64
dtype: object

In [11]:
## Replace the zero values in the DataFrame to NaN.
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

In [12]:
## Counting the Number of NaNs in the DataFrame
count_nan = len(df) - df.count()
count_nan

Book Name          0
Author             1
Rating             0
Customers_Rated    8
Price              0
dtype: int64

**From the above output: there is a total of six books that do not have an Author Name, while one book does not have a price associated with it**

In [13]:
## Let's drop these NaNs.
df = df.dropna()

### Authors Highest Priced Book
**Let's find out which all authors had the highest-priced book. You will visualize the results for such the top 20 authors.**

In [14]:
data = df.sort_values(["Price"], axis=0, ascending=False)[:15]
data

,Book Name,Author,Rating,Customers_Rated,Price
21,Demon Slayer Complete Box Set: Includes volume...,Koyoharu Gotouge,4.6,103.0,294
42,The Lyrics: 1956 to the Present,Paul McCartney,4.8,203.0,99
41,Harry Potter Paperback Box Set (Books 1-7),J. K. Rowling,4.9,57791.0,45
90,Fizban's Treasury of Dragons (Dungeon & Dragon...,Wizards RPG Team,4.7,606.0,29
2,The 1619 Project: A New Origin Story,Nikole Hannah-Jones,4.4,63.0,24
38,The Dawn of Everything: A New History of Humanity,David Graeber,4.4,71.0,22
63,Dwyane,Dwyane Wade,3.6,5.0,22
98,Woke Up This Morning: The Definitive Oral Hist...,Michael Imperioli,4.4,161.0,21
15,Humans,Brandon Stanton,4.9,15618.0,21
4,Welcome to Dunder Mifflin: The Ultimate Oral H...,Brian Baumgartner,5.0,6.0,20


# <center>Data Visualization</center>

In [15]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()

Loading BokehJS ...

In [16]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

In [17]:
show(p)

**From the above graph: the top three highest-priced books are by the author `Koyohau Gotouge`, `Paul McCartney` and `J.K.Rowling`.
**
### Top Rated Books and Authors wrt Customers Rated
 
**which authors have the top-rated books and which books of those authors are top rated**

In [18]:
data = df[df['Customers_Rated'] > 1000]
data = data.sort_values(['Rating'],axis=0, ascending=False)[:15]
data

,Book Name,Author,Rating,Customers_Rated,Price
56,Polar Express 30th anniversary edition,Chris Van Allsburg,4.9,14138.0,11
34,Cat Kid Comic Club: Perspectives: A Graphic No...,Dav Pilkey,4.9,13712.0,10
30,Little Blue Truck's Christmas,Alice Schertle,4.9,11989.0,10
77,Joshua Weissman: An Unapologetic Cookbook. #1 ...,Joshua Weissman,4.9,3167.0,17
26,The Very Hungry Caterpillar,Eric Carle,4.9,42110.0,5
41,Harry Potter Paperback Box Set (Books 1-7),J. K. Rowling,4.9,57791.0,45
24,I Love You to the Moon and Back,Amelia Hepworth,4.9,43692.0,4
57,Dog Man: Mothering Heights: A Graphic Novel (D...,Dav Pilkey,4.9,15472.0,4
59,Super Extra Deluxe Essential Handbook (Pokémon...,Scholastic,4.9,1399.0,12
82,Penguin Random House The Wonderful Things You ...,Emily Winfield Martin,4.9,18261.0,8


In [19]:
p = figure(x_range=data.iloc[:,0], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,0], top=data.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2
show(p)

In [20]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2
show(p)

In [21]:
data = df.sort_values(["Customers_Rated"], axis=0, ascending=False)[:20]

In [22]:
data

,Book Name,Author,Rating,Customers_Rated,Price
76,The Midnight Library: A Novel,Matt Haig,4.3,117648.0,13
37,"The Boy, the Mole, the Fox and the Horse",Charlie Mackesy,4.8,86426.0,13
28,The Four Agreements: A Practical Guide to Pers...,Don Miguel Ruiz,4.7,68134.0,5
32,My First Learn to Write Workbook: Practice for...,Crystal Radke,4.8,63347.0,4
41,Harry Potter Paperback Box Set (Books 1-7),J. K. Rowling,4.9,57791.0,45
88,The Subtle Art of Not Giving a F*ck: A Counter...,Mark Manson,4.6,57438.0,11
16,If Animals Kissed Good Night,Ann Whitford Paul,4.8,57434.0,4
68,School Zone - Big Preschool Workbook - 320 Pag...,School Zone,4.8,56006.0,3
9,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,4.8,55828.0,11
55,The Last Thing He Told Me: A Novel,Laura Dave,4.3,52036.0,9


In [23]:
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.palettes import d3
#colors has a list of colors which can be used in plots
colors = itertools.cycle(palette)

palette = d3['Category20'][20]

In [24]:
index_cmap = factor_cmap('Author', palette=palette,
                         factors=data["Author"])

In [25]:
p = figure(plot_width=700, plot_height=700, title = "Top Authors: Rating vs. Customers Rated")
p.scatter('Rating','Customers_Rated',source=data,fill_alpha=0.6, fill_color=index_cmap,size=20,legend='Author')
p.xaxis.axis_label = 'RATING'
p.yaxis.axis_label = 'CUSTOMERS RATED'
p.legend.location = 'top_left'

In [26]:
show(p)